# Colocando o Istio em execução

## Acessando o k8s

Vamos configurar o ambiente, você precisará configurá-lo a cada nova seção ou executar o jupyter passando as variáveis, como descrito no [README](README.md) e não precisará repetir esses comandos novamente.

In [ ]:
# [Opcional] Se você não estiver usando o docker-desktop será necessário obter o arquivo de configuração e ajustar a variável KUBECONFIG
export KUBECONFIG=~/.kube/config # Local do kubeconfig do docker-desktop. Selecione o local onde você colocou o arquivo de config para outras opções (AKS, EKS, GKE, etc).

In [ ]:
# [Opcional] Se você tem mais de um cluster na configuração, verifique se está apontando para o correto.
kubectl config get-contexts

In [ ]:
# Vamos verificar como estão os nós do nosso cluster:
kubectl get nodes

## Instalando o Istio (linux)

Para instalar a última versão do Istio, neste momento 1.10.3, você pode ir até a página [Getting Started](https://istio.io/latest/docs/setup/getting-started/#download) ou seguir as instruções abaixo:

In [ ]:
curl -L https://istio.io/downloadIstio | sh -

Para usar o comando `istioctl`, que está no diretório `bin` do download, coloque-o na variável `PATH` ou copie o arquivo `bin/istioctl` para um diretório no seu `PATH`

Copiando o istioctl para o diretório de binários

O istioctl é um utilitário de linha de comando e foi copiado para a sua máquina durante o download acima.

Para utilizá-lo, basta colocar o arquivo em algum diretório listado na variável PATH.

O istioctl é um utilitário de linha de comando e foi copiado para a sua máquina durante o download acima.

Para utilizá-lo, basta colocar o arquivo em algum diretório listado na variável PATH.

In [ ]:
echo $PATH

Os diretórios mais indicados são os do usuário para binários, caso você não tenha, pode criar um e colocá-lo na variável PATH.

A seguir criaremos o diretório `˜/bin`, adicionaremos o caminho na variável PATH e copiaremos o arquivo para o diret

In [ ]:
ISTIO_VERSION=1.10.3
# Verificar onde está o executável do Istio
ls istio-$ISTIO_VERSION/bin
export PATH=$PWD/istio-$ISTIO_VERSION/bin:$PATH

In [ ]:
# Teste
istioctl version

Você também pode copiar o comando em qualquer diretório ou mantê-lo no diretório do Istio e ajustar a variável `PATH` na sessão.

Para tornar essa configuração permanente, você deve adicionar o caminho absoluto (substitua p PWD) a variável PATH no arquivo de configuração do shell do seu usuário, por exemplo ~/.bashrc ou ~/.profile ou ~/.zshrc, etc.

## O mínimo do Istio

Desde a versão 1.6 o Istio é composto de uma única entrega chamada `istiod` e ela pode ser instalada com o comando abaixo:

In [ ]:
# Isso deve demorar de 2 a 5 minutos
istioctl install --set profile=minimal --skip-confirmation

> Em Mar-2021 não é possível instalar o Istio no GKE Autopilot, ele retorna o erro: `Istiod encountered an error: failed to update resource with server-side apply for obj MutatingWebhookConfiguration`. Veja mais sobre [incopatibilidades do GKE Autopilot](https://cloud.google.com/kubernetes-engine/docs/concepts/autopilot-overview?hl=pt-br#unsupported_cluster_features).
> Para GKE crie um cluster padrão Zonal.

Vamos verificar o que foi instalado

In [ ]:
# Obtendo os namespaces
kubectl get ns

# Verificando o namespace do Istio
kubectl get all -n istio-system

Instalamos apenas o plano de controle e mais adiante instalaremos o ingress, egress e alguns adicionais como grafana, kiali e jaeger, mas já temos o Istio instalado e pronto para gerenciar os Envoys.

## Istio CRDs

O Istio é uma aplicação que é executada no kubernetes e adiciona recursos personalizados ([CRD](https://kubernetes.io/docs/concepts/extend-kubernetes/api-extension/custom-resources/)) para sua configuração.

Você pode obter uma lista completa executando o comando abaixo:

In [ ]:
kubectl api-resources --api-group=networking.istio.io

kubectl api-resources --api-group=security.istio.io

Esses são os recursos adicionados e na segunda coluna o nome abreviado, você pode utilizar um ou outro, por exemplo:

In [ ]:
# O comando
kubectl get svc

# É equivalente ao

kubectl get services

> Note que todos as configurações para recursos do Istio são aplicados para um _namespace_ (Namespaced=true), isso significa que a configuração terá efeito apenas em um _namespace_, porém, configurações que poderão ser aplicadas para toda a malha de serviços serão realizadas no _namespace_ do Istio, o `istio-system`.

Você verá muito disso no kubernetes, para uma lista completa de recursos e suas abreviações execute o mesmo comando acima, mas sem o filtro `kubectl api-resources`

In [ ]:
kubectl api-resources

> **Dica pro**: Quem utiliza o kubectl com muita frequência costuma criar apelidos para os comandos mais comuns, por exemplo:

In [ ]:
# Alias para o comando kubectl
alias k=kubectl
# Teste
k get svc
# Alias para o comando kubectl get
alias kg='kubectl get'
# Teste
kg svc

> Você pode colocar esses comandos nos arquivos de configuração do shell (.profile, .bashrc, etc), para torná-los persistentes entre sessões.

## Ativando o Istio para um namespace

Um elemento chave do Istio é o _sidecar_ ou _proxy_, como veremos na discussão sobre arquitetura do Istio. Para que o Istio injete-o nos PODs da aplicação, é necessário marcar o _namespace_ da seguinte forma:

In [ ]:
kubectl label namespace default istio-injection=enabled

Vamos verificar como ficou a configuração do namespace

In [ ]:
kubectl describe ns/default

O `istiod` irá monitorar todos os _namespaces_ com o rótulo `istio-injection=enabled` e adicionar ao POD um _container_ de proxy, o _sidecar_.

Vamos fazer _deploy_ de uma aplicação exemplo para verificar esse comportamento.

## Criando uma aplicação demo

Vamos criar uma aplicação simples.

In [ ]:
# Vamos criar um diretório para nossos exemplos:
mkdir -p assets/exemplos/simple-app

# O arquivo de Deployment:
cat <<EOT > assets/exemplos/simple-app/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: simple-app
  labels:
    app: simple-app
    version: v1
spec:
  replicas: 1
  selector:
    matchLabels:
        app: simple-app
        version: v1
  template:
    metadata:
      labels:
        app: simple-app
        version: v1
    spec:
      containers:
        - name: simple-app
          image: "nginx:stable"
          imagePullPolicy: IfNotPresent
          ports:
            - name: http
              containerPort: 80
              protocol: TCP
EOT

# O de serviço
cat <<EOT > assets/exemplos/simple-app/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: simple-app
  labels:
    app: simple-app
    version: v1
spec:
  type: ClusterIP
  ports:
    - port: 80
      targetPort: http
      protocol: TCP
      name: http
  selector:
    app: simple-app
    version: v1
EOT

In [ ]:
# Listando os arquivos criados:
ls -la assets/exemplos/simple-app

Criamos dois arquivos, o `deployment.yaml` e o `service.yaml` no diretório `assets/exemplos/simple-app`, agora vamos instala-la no _namespace_ default (quando omitido é onde os recursos serão criados).

Inspecione os arquivos e tente descobrir o que será instalado no cluster, uma dica, procure a pela imagem.

In [ ]:
kubectl apply -f assets/exemplos/simple-app

Verificando a situação do POD.

In [ ]:
kubectl get pods

> Para instalar a aplicação em um _namespace_ diferente adicione `--namesapce` ou `-n`ao comando. Exemplo: `kubectl apply -f exemplos/simple-app -n test-app`

Vamos acessar nossa aplicação, ela foi configurada para o tipo de serviço `ClusterIP`, o que significa que o acesso é interno, apenas entre os PODs do cluster, mas podemos acessá-la utilizando o comando `kubectl port-forward`.

In [ ]:
kubectl port-forward svc/simple-app 8000:80

Vá até o seu navegador e entre acesse a url: <http://localhost:8000>

Pronto, você tem acesso à sua aplicação como se estivesse sendo executada na sua máquina. Claro que o kubernetes pode estar na sua máquina, mas isso funcionará em qualquer kubernetes, local ou remoto.

Para interromper:
* No Jupyter Lab ou Notebook: Clique no icone <kbd>◾</kbd> (_Interrup the kernel_ na barra de ferramentas)
* No terminal: tecle <kbd>CTRL</kbd>+<kbd>C</kbd>

Verificando o que foi instalado.

In [ ]:
kubectl get --show-labels all

Note que o pod/simple-app tem 2/2 na coluna pronto (Ready), isso significa que dois containers de dois estão ok, vamos verificar quem é o segundo container.

In [ ]:
# Usaremos um dos labels do pod para encontra-lo
kubectl describe pod -l app=simple-app

É uma grande quantidade de informação, vamos procurar uma seção chamada `Containers` e nela a nossa aplicação `simple-app`.

Como você pode ver, a imagem desse container é `nginx`, com a tag `stable`, mais abaixo tem um segundo container `istio-proxy`, com a imagem `docker.io/istio/proxyv2` e a _tag_ para a  versão `1.10.3`.

Esse container não faz parte do [assets/exemplos/simple-app/deployment.yaml](exemplos/2_simple-app/deployment.yaml), ele foi adicionado ao seu pod pelo `istiod`.

Caso você precise saber todos os _namespaces_ que tem a injeção do _conteiner_ _proxy_ do Istio ativado, basta executar o comando:

In [ ]:
kubectl get ns -l istio-injection=enabled

kubectl get pods

Até o momento, somente o namespace `default`, onde configuramos a marcação, foi encontrado.

## Conclusão

Neste ponto temos o mínimo do Istio em execução no nosso cluster, mas com exceção de um container extra isso não significa muito.

Na próxima parte iremos explorar os recursos que esta instalação mínima do Istio pode oferecer.